## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kabalo,CD,-6.0500,26.9167,69.84,95,100,1.25,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.02,72,100,8.16,overcast clouds
2,2,Marawi,PH,7.9986,124.2928,71.65,88,94,1.19,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,51.91,50,0,6.91,clear sky
4,4,Havelock,US,34.8791,-76.9013,33.78,56,0,5.01,clear sky


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_city_df.sample(5)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
39,39,Faanui,PF,-16.4833,-151.7500,80.64,72,43,9.08,light rain
312,312,Qui Nhon,VN,13.7667,109.2333,71.89,87,96,5.82,overcast clouds
684,684,Pitimbu,BR,-7.4706,-34.8086,77.59,80,67,7.43,broken clouds
540,540,Wad Madani,SD,14.4012,33.5199,66.94,20,3,12.86,clear sky
599,599,Monrovia,LR,6.3005,-10.7969,79.34,83,63,5.32,broken clouds


In [51]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

Unnamed: 0             268
City                   268
Country                268
Lat                    268
Lng                    268
Max Temp               268
Humidity               268
Cloudiness             268
Wind Speed             268
Current Description    268
dtype: int64

In [77]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city_df.dropna()
#preferred_city_df.dropna(inplace=True)
# preferred_city_df.isnull().sum()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kabalo,CD,-6.0500,26.9167,69.84,95,100,1.25,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.02,72,100,8.16,overcast clouds
2,2,Marawi,PH,7.9986,124.2928,71.65,88,94,1.19,overcast clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,75.02,78,39,6.64,scattered clouds
13,13,Lucapa,AO,-8.4192,20.7447,66.24,95,100,2.89,light rain
...,...,...,...,...,...,...,...,...,...,...
680,680,Soyo,AO,-6.1349,12.3689,78.37,87,99,4.14,light rain
682,682,Fare,PF,-16.7000,-151.0167,80.58,72,30,9.17,light rain
684,684,Pitimbu,BR,-7.4706,-34.8086,77.59,80,67,7.43,broken clouds
686,686,Olutanga,PH,7.3106,122.8464,80.96,74,97,4.38,overcast clouds


In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kabalo,CD,69.84,overcast clouds,-6.0500,26.9167,
1,Rikitea,PF,77.02,overcast clouds,-23.1203,-134.9692,
2,Marawi,PH,71.65,overcast clouds,7.9986,124.2928,
10,Puerto Ayora,EC,75.02,scattered clouds,-0.7393,-90.3518,
13,Lucapa,AO,66.24,light rain,-8.4192,20.7447,
16,Mar Del Plata,AR,77.02,clear sky,-38.0023,-57.5575,
17,Salalah,OM,68.09,clear sky,17.0151,54.0924,
18,Hobart,AU,77.02,broken clouds,-42.8794,147.3294,
20,Busselton,AU,73.09,clear sky,-33.6500,115.3333,
23,Majene,ID,79.84,overcast clouds,-3.5403,118.9707,


In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping city")
        
hotel_df.head(10)        

Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kabalo,CD,69.84,overcast clouds,-6.0500,26.9167,
1,Rikitea,PF,77.02,overcast clouds,-23.1203,-134.9692,People ThankYou
2,Marawi,PH,71.65,overcast clouds,7.9986,124.2928,Derogongan Residence
10,Puerto Ayora,EC,75.02,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Lucapa,AO,66.24,light rain,-8.4192,20.7447,Residência Ulisses Dos Santos
16,Mar Del Plata,AR,77.02,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
17,Salalah,OM,68.09,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
18,Hobart,AU,77.02,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
20,Busselton,AU,73.09,clear sky,-33.6500,115.3333,Observatory Guest House
23,Majene,ID,79.84,overcast clouds,-3.5403,118.9707,Kost Putri RABBANI


In [78]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] !=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,77.02,overcast clouds,-23.1203,-134.9692,People ThankYou
2,Marawi,PH,71.65,overcast clouds,7.9986,124.2928,Derogongan Residence
10,Puerto Ayora,EC,75.02,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Lucapa,AO,66.24,light rain,-8.4192,20.7447,Residência Ulisses Dos Santos
16,Mar Del Plata,AR,77.02,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
...,...,...,...,...,...,...,...
680,Soyo,AO,78.37,light rain,-6.1349,12.3689,Hotel Mpampa
682,Fare,PF,80.58,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
684,Pitimbu,BR,77.59,broken clouds,-7.4706,-34.8086,Reserva do Abiaí
686,Olutanga,PH,80.96,overcast clouds,7.3106,122.8464,Morales Lodge


In [79]:
# 8a. Create the output File (CSV)
output_data_file = "VacationPy_Search.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [80]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [82]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))